In [43]:
# Implementation of the Eisenberg and Noe 2001 Debt Model in Python
# Eisenberg and Noe 2001 analyse the properties of intercorporate cash flows in financial systems featuring cyclical 
# interdependence and endogenously determined clearing vectors. The model computes clearing vectors for interlinked 
# financial systems. A clearing vector is a vector of payments from nodes in the financial system to other nodes and 
# must satisfy the conditions of proportional repayments of liabilities in default (we assume all debt claims have 
# equal priority), limited liability of equity and absolute priority of debt over equity. The clearing vector is 
# computed through a "fictitious sequential default" algorithm in which the set of defaulting firms at the start of 
# each round is fixed by the dynamic adjustments of the system from the preceding round. In each new round, an attempt
# to clear the system that assumes that only nodes that defaulted in the previous round default. If no new defaults 
# occur, the algorithm terminates (i.e. a node is a distinct economic entity or a financial node i.e. a firm). 

# This algorithm gives the clearing vector and a natural measure of systemic risk (i.e. the exposure of a given node
# in the system to defaults by other firms - this is based on the number of waves of defaults required to 
# induce a given firm in the system to fail)

# Julian Kanjere, knjjul001@myuct.ac.za, September 2021

######### IMPORTS ######### 

import numpy as np
import pandas as pd
import random

######### /IMPORTS ######### 

######### MODEL SETUP ######### 

NUM_AGENTS = 3 # i.e. n x n assuming a star network with each node connected to the other i.e. cyclical interdependence
ALPHABET ='ABCDEFGHIJKLMNOPQRSTUVWXYZ' 
AGENT_LABELS = [ALPHABET[i] for i in range(0, NUM_AGENTS)] 
print("\n--------------------------------------------------------\n")
print("Agent labels", str(AGENT_LABELS))
print("\n--------------------------------------------------------\n")

MATRIX_SIZE = NUM_AGENTS * NUM_AGENTS # e.g. 5 x 5

# Nominal liability ranges used when randomly initialising the NOMINAL_LIABILITY_MATRIX
NOMINAL_LIABILITY_LOWER_RANGE = 0
NOMINAL_LIABILITY_UPPER_RANGE = 10

# Exogenous cashflow ranges used when randomly initialising the OPERATING_CASH_FLOW_VECTOR
EXOGENOUS_CASHFLOW_LOWER_RANGE = 0
EXOGENOUS_CASHFLOW_UPPER_RANGE = 10


# n x n nominal liabilities matrix L captures the nominal liability of one node to another in the system
# i.e. L_ij is nominal liability of node i to node j 
# NOMINAL_LIABILITY_MATRIX = np.zeros((NUM_AGENTS, NUM_AGENTS))
NOMINAL_LIABILITY_MATRIX = np.random.randint(NOMINAL_LIABILITY_LOWER_RANGE, NOMINAL_LIABILITY_UPPER_RANGE, 
                                             size=MATRIX_SIZE).reshape(NUM_AGENTS, NUM_AGENTS)
np.fill_diagonal(NOMINAL_LIABILITY_MATRIX, 0) # a node cannot have liabilities against itself
NOMINAL_LIABILITY_MATRIX_DF = pd.DataFrame(NOMINAL_LIABILITY_MATRIX, index=AGENT_LABELS, columns=AGENT_LABELS)
print("\nNOMINAL LIABILITY MATRIX Data Frame (i.e. what node i expects to pay node j):")
display(NOMINAL_LIABILITY_MATRIX_DF)

print("\nNominal liabilities for each node:")
for row in range(0, NUM_AGENTS):
    for column in range(0, NUM_AGENTS):
        if row == column:
            pass
        else:
            print("Liability of Node %s to Node %s is %s" % (AGENT_LABELS[row], AGENT_LABELS[column], 
                                                             NOMINAL_LIABILITY_MATRIX_DF.iloc[row, column]))
    print("\n")

# used to show what each agent is receiving, read row wise
print("\n NOMINAL LIABILITY MATRIX TRANSPOSED Data Frame (i.e. what node j expects to receive from i):")                    
NOMINAL_LIABILITY_MATRIX_TRANSPOSED_DF = NOMINAL_LIABILITY_MATRIX_DF.T 

display(NOMINAL_LIABILITY_MATRIX_TRANSPOSED_DF)
for row in range(0, NUM_AGENTS):
    for column in range(0, NUM_AGENTS):
        if row == column:
            pass
        else:
            print("Node %s expects to receive %s from Node %s" % 
                  (AGENT_LABELS[row], NOMINAL_LIABILITY_MATRIX_TRANSPOSED_DF.iloc[row, column],
                  AGENT_LABELS[column]))
    print("\n")
    
# n x n relative liabilities matrix which represents the nominal liability of one node to another in the system
# as a proportion of the debtor nodes total liabilites i.e. L_ij / p_i. We first initialise with zeros. 
RELATIVE_LIABILITY_MATRIX = np.zeros((NUM_AGENTS, NUM_AGENTS)) 
RELATIVE_LIABILITY_MATRIX_DF = pd.DataFrame(RELATIVE_LIABILITY_MATRIX, index=AGENT_LABELS, columns=AGENT_LABELS)

# a dictionary whose key is the round starting from 1 and value is a list of exogenous cash infusion to a 
# node i.e. from outside sources
OPERATING_CASH_FLOW_VECTOR = []
print("\n--------------Operating Cash Flow Vector:----------------\n")
for row in range (0, NUM_AGENTS):
    exogenous_cash_flow = random.randint(EXOGENOUS_CASHFLOW_LOWER_RANGE, EXOGENOUS_CASHFLOW_UPPER_RANGE)
    OPERATING_CASH_FLOW_VECTOR.append(exogenous_cash_flow)
    print("Exogenous cash flow for Node %s: %s" % (AGENT_LABELS[row], str(exogenous_cash_flow)))
print(OPERATING_CASH_FLOW_VECTOR)

# TODO - alternative to vectors as a dict of lists is to simply use dataframes?

# a dictionary whose key is the round starting from 1 and value is a list or a set of 
# total payments made by each node to other nodes in the system i.e. p = (p_1, p_2, p_3, ...)
TOTAL_DOLLAR_PAYMENT_VECTOR = {}

# a dictionary whose key is the round starting from 1 and value is a list of payments 
# made by each node to other nodes in the system i.e. p_bar = (p_bar_1, p_bar_2, p_bar_3, ...)
TOTAL_OBLIGATION_VECTOR = {} 

# a dictionary whose key is the round starting from 1 and value is a list of payments 
CLEARING_PAYMENT_VECTOR = {} 

# a dictionary whose key is the round starting from 1 and value is a list of defaulters for that round
DEFAULTERS_VECTOR = {} 

######### /MODEL SETUP ######### 

######### HELPER FUNCTIONS ######### 

def bool_limited_liability(i):
    '''Function to check that the limited liability for a node i holds i.e. the payment made by the node is
    less than or equal to the sum of the payments received by the node plus the exogenous operating cash flow 
    i.e. TOTAL_DOLLAR_PAYMENT_VECTOR (i.e. p_i) value is less than or equal to calculate_total_cash_flow_for_node()'''
    pass

def bool_proportional_repay():
    pass

def bool_debt_over_equity(i):
    '''Function to check the absolute priority rule for a node i i.e. either obligations are paid in full or all avaialable 
    cash flow (i.e. sum of the payments received by the node plus the exogenous operating cash flow) is paid to creditors for a node i holds i.e. the payment made by the node is
    less than or equal to the sum of the payments received by the node plus the exogenous operating cash flow
    i.e. first compare TOTAL_DOLLAR_PAYMENT_VECTOR value (i.e. p_i) to the TOTAL_OBLIGATION_VECTOR value (i.e. p_hat_i)
    to establish whether obligations are paid in full. If p_i < p_hat_i, then:
    if calculate_total_cash_flow_for_node() - proposed payment (you could think p_i) > 0, 
    you fail the absolute priority condition and advise that payment p_i should be = calculate_total_cash_flow_for_node()'''
    pass

def bool_check_defaults(round):
    '''Function to check whether there are any defaults in a given round. If not, algorithm can terminate'''
    pass

def bool_check_relative_liabilities_matrix():
    '''Function to sanity check that the sum of the proportions in the RELATIVE_LIABILITY_MATRIX add up to 1
    i.e. Further, all payments are made to some node in the system, therefore for all nodes, the sum the of the
    proportions should equal 1.'''
    pass

def bool_check_clearing_payment_vector(r):
    '''Function to sanity check clearing payment vector for a round r for each node i. 
    Each node pays a minimum of either calculate_total_cash_flow_for_node() or TOTAL_OBLIGATION_VECTOR value (i.e. p_hat_i).'''
    pass

def return_operating_cash_flow_for_node(i):
    '''Function to return operating cash flow to node i at the start of the observation. The operating cash flow is 
    the exogenous operating cash flow received by node i'''
    return float(OPERATING_CASH_FLOW_VECTOR[i])

def return_single_nominal_liability_for_node(i, j):
    '''Function to return nominal liability from node i to j (where i is is dataframe index and j is dataframe row)'''
    return float(NOMINAL_LIABILITY_MATRIX_DF.iloc[i, j])

def return_single_payment_in_for_node(i, j):
    '''Function to return payment received by node i from j (where i is is dataframe index and j is dataframe row)'''
    return float(NOMINAL_LIABILITY_MATRIX_TRANSPOSED_DF.iloc[i, j])

def return_single_relative_liability_for_node(i, j):
    '''Function to return relative liability from node i to j (where i is is dataframe index and j is dataframe row)'''
    return float(RELATIVE_LIABILITY_MATRIX_DF.iloc[i, j])

def return_single_relative_payment_in_for_node(i, j):
    '''Function to return relative payment received by node i from j (where i is is dataframe index and j is dataframe row)'''
    return float(RELATIVE_LIABILITY_MATRIX_TRANSPOSED_DF.iloc[i, j])

def return_total_dollar_payment_by_node(i,round):
    '''Function to return total dollar payment by node i (NB - this is not total obligation) i.e p_i'''
    if round not in TOTAL_DOLLAR_PAYMENT_VECTOR:
        print("Error occured, total dollar payment vector not found for current round %s. Will now abort." % 
              str(round))
        sys.exit(1)
    
    total_dollar_vector = TOTAL_DOLLAR_PAYMENT_VECTOR[round]
    total_dollar_payment_by_i = total_dollar_vector[i]
    print("Total dollar payment by Node %s (i.e. p_i) is %s\n" % (AGENT_LABELS[i], total_dollar_payment_by_i))

    return float(total_dollar_vector[i])

def calculate_total_nominal_liabilities_out_for_node(i, round):
    '''Function to calculate nominal liabilities out for node i in a given round i.e. debtor node's total liabilities 
    which is p_hat_i i.e. for j = 1 upto n, calculate the sum of L_ij. These nominal liabilities represent the promised payments due to other nodes in the system. The 
    inidividual nominal liabilities are taken from NOMINAL_LIABILITY_MATRIX and the total nominal obligations for 
    each node are stored in the TOTAL_OBLIGATION_VECTOR. 
    '''
    total_liability = 0
    
    if round not in TOTAL_OBLIGATION_VECTOR:
        TOTAL_OBLIGATION_VECTOR[round] = []
    
    obligation_vector = TOTAL_OBLIGATION_VECTOR[round]
    
    # where i is the row of index of the dataframe which identifies the node
    print("\n----------------Liabilities for Node %s------------------\n" % AGENT_LABELS[i])
    for column in range(0, NUM_AGENTS):
        if i == column:
            pass
        else:
            nominal_liability_i_j = return_single_nominal_liability_for_node(i, column)
            print("Liability of Node %s to Node %s (i.e. P_ij) is %s" % (AGENT_LABELS[i], AGENT_LABELS[column], 
                                                             str(nominal_liability_i_j)))
            total_liability = total_liability + nominal_liability_i_j
            
    
    TOTAL_OBLIGATION_VECTOR[round].append(total_liability)
    print("Total nominal liabilities for Node %s (i.e. p_hat_i) is %s\n" % (AGENT_LABELS[i], total_liability))
    # print(TOTAL_OBLIGATION_VECTOR)
    return total_liability


def calculate_total_relative_liabilities_out_for_node(i, round):
    '''Function to calculate relative liabilities out for node i in a given round i.e. the nominal liability of one 
    node to another in the system as a proportion of the debtor node's total liabilities i.e. Pi_ij = L_ij/p_hat_i.
    These nominal liabilities represent the promised payments due to other nodes in the system. 
    This is represented in the nominal liabilities matrix RELATIVE_LIABILITY_MATRIX. 
    '''
    p_bar_i = calculate_total_nominal_liabilities_out_for_node(i, round)
    sum_relative_liability = 0
    
    # where i is the row of index of the dataframe
    for column in range(0, NUM_AGENTS):
        if i == column:
            pass
        else:
            nominal_liability_i_j = return_single_nominal_liability_for_node(i, column)
            relative_liability_i_j =  nominal_liability_i_j/p_bar_i
            print("Relative Liability of Node %s to Node %s is %s" % (AGENT_LABELS[i], AGENT_LABELS[column], 
                                                             relative_liability_i_j))
            RELATIVE_LIABILITY_MATRIX_DF.iloc[i, column] = relative_liability_i_j
            sum_relative_liability = sum_relative_liability + relative_liability_i_j
    
    return sum_relative_liability

                  
def calculate_total_payments_in_for_node(i, round):
    '''Function to calculate total cash flow to node i in a given round which is payments received (what the node 
    receives from other nodes i.e. endogenous) plus operating cashflow (exogenous)'''
    total_payments_in = 0
    exogenous_operating_cash_flow = OPERATING_CASH_FLOW_VECTOR(i) #for a node i
    
    # where i is the row of index of the dataframe which identifies the node and column is the j
    print("\n----------------Relative Payments in for Node %s------------------\n" % AGENT_LABELS[i])
    for column in range(0, NUM_AGENTS):
        if i == column:
            pass
        else:
            single_relative_payment_i_j = return_single_relative_payment_in_for_node(i, column)
            print("Relative Payment in to Node %s from Node %s is %s" % (AGENT_LABELS[i], AGENT_LABELS[column], 
                                                             str(single_relative_payment_i_j)))
            p_j = return_total_dollar_payment_by_node(column)
            payment_i_j = single_relative_payment_i_j * p_j
            total_payments_in = total_payments_in + single_relative_payment_i_j
    
    print("Total payments in to Node %s is %s" % (AGENT_LABELS[i], str(total_payments_in)))
                  
    return total_payments_in
                  
def calculate_total_cash_flow_for_node(i, round):
    '''Function to calculate total cash flow to node i in a given round which is payments received (what the node 
    receives from other nodes i.e. endogenous) plus operating cashflow (exogenous)'''
    operating_cashflow = return_operating_cash_flow_for_node(i) #this is only relevant for round 1 i.e. observed at start
    total_payments_in = calculate_total_payments_in_for_node(i, round)
    total_cash_flow = operating_cashflow + total_payments_in
    return total_cash_flow
                  
def calculate_total_equity_for_node(i, payments_in, liabilities_out):
    '''Function to calculate total equity of node i in a given round which is total cash flow (i.e. payments received 
    (endogenous) plus operating cashflow (exogenous)) minus liabilities out (i.e. nominal payments)
    '''
    total_cash_flow = calculate_total_cash_flow_for_node(i)
    return total_cash_flow - liabilities_out
                  
def calculate_total_value_for_node(i, equity, liabilities_out):
    '''Function to calculate total value of node i in a given round which is debt plus equity'''
    return total_cash_flow - liabilities_out

def calculate_systemic_risk(r):
    '''Function to calculate systemic risk for each node in a given round i.e. this is based on the number of 
    waves of defaults required to induce a given node in the system to fail'''
    pass

def calculate_total_obligation_vector(r):
    '''Function to return the total obligation vector made of the obligations for each node
    in a round r. i.e. p_bar = (p1_bar, p2_bar,...,pn_bar). This vector represents the payment level required for 
    complete satisfaction of all contractual liabilities by all nodes. This will loop over all n nodes and
    return the output from calculate_total_nominal_obligation_for_node()'''
    pass

def calculate_operating_cashflow_vector(r):
    '''Function to return the operating cashflow vector comprised of exogenous operating cash flow
    received by each node in a given round r'''
    pass

def calculate_clearing_payment_vector(r):
    '''Function to return the clearing payment vector in a given round r. For each node i, we maximise the payment p
    which is the range [0, TOTAL_OBLIGATION_VECTOR i.e. p_hat_i] subject to p <= calculate_total_cash_flow_for_node().
    We then need to confirm that limited_liability and absolute priority are satisfied.'''
    pass

def update_nominal_liabilities_matrix(r):
    '''Function to initialise or update the nominal liabilities matrix L in a given round r. All nominal claims are 
    nonnegative (i.e. L_ij > 0) and no node has a nominal claim against itself (i.e. L_ii = 0).'''
    pass

def update_relative_liabilities_matrix(r):
    '''Function to initialise or update the relative liabilities matrix Pi in a given round r. This matrix captures
    the nominal liability of one node to another in the system as a proportion of the debtor node's total
    liabilities. After this is updated, you can call bool_check_relative_liabilities_matrix() to sanity check
    that the entries add up to 1'''
    pass
######### /HELPER FUNCTIONS ######### 
print("\n--------------------------------------------------------\n")

round = 1
print("Calculating relative liabilities for each node...\n")
for row in range(0, NUM_AGENTS):
    sum_relative_liability = calculate_total_relative_liabilities_out_for_node(row, round)
    print("Sum of Relative Liabilities for Node %s is %s" % (AGENT_LABELS[row], str(sum_relative_liability)))
    
print("\n--------------------------------------------------------\n")
print("\nRELATIVE LIABILITY MATRIX Data Frame:")
display(RELATIVE_LIABILITY_MATRIX_DF)

print("\n--------------------------------------------------------\n")                
# used to show what each agent is receiving in relative terms, read row wise
print("\nRELATIVE LIABILITY MATRIX TRANSPOSED Data Frame (i.e. what node i expects to " \
      "receive from j in relative terms):")                    
RELATIVE_LIABILITY_MATRIX_TRANSPOSED_DF = RELATIVE_LIABILITY_MATRIX_DF.T 

display(RELATIVE_LIABILITY_MATRIX_TRANSPOSED_DF)
for row in range(0, NUM_AGENTS):
    for column in range(0, NUM_AGENTS):
        if row == column:
            pass
        else:
            print("Node %s expects to receive %s from Node %s" % 
                  (AGENT_LABELS[row], RELATIVE_LIABILITY_MATRIX_TRANSPOSED_DF.iloc[row, column],
                  AGENT_LABELS[column]))
    print("\n")
                  
print("\n--------------------------------------------------------\n")
print("\nTOTAL OBLIGATION VECTOR (i.e. total nominal obligations for each node i.e. p_hat_i):")
obligation_vector_for_round = TOTAL_OBLIGATION_VECTOR[round]
for row in range (0, NUM_AGENTS):
    print("Total nominal obligation for Node %s (i.e. p_hat_%s): %s" % (AGENT_LABELS[row],
                                                                        str(row + 1),
                                                                        str(obligation_vector_for_round[row])))
print(TOTAL_OBLIGATION_VECTOR)




--------------------------------------------------------

Agent labels ['A', 'B', 'C']

--------------------------------------------------------


NOMINAL LIABILITY MATRIX Data Frame (i.e. what node i expects to pay node j):


,A,B,C
A,0,6,5
B,6,0,9
C,4,2,0



Nominal liabilities for each node:
Liability of Node A to Node B is 6
Liability of Node A to Node C is 5


Liability of Node B to Node A is 6
Liability of Node B to Node C is 9


Liability of Node C to Node A is 4
Liability of Node C to Node B is 2



 NOMINAL LIABILITY MATRIX TRANSPOSED Data Frame (i.e. what node j expects to receive from i):


,A,B,C
A,0,6,4
B,6,0,2
C,5,9,0


Node A expects to receive 6 from Node B
Node A expects to receive 4 from Node C


Node B expects to receive 6 from Node A
Node B expects to receive 2 from Node C


Node C expects to receive 5 from Node A
Node C expects to receive 9 from Node B



--------------Operating Cash Flow Vector:----------------

Exogenous cash flow for Node A: 1
Exogenous cash flow for Node B: 5
Exogenous cash flow for Node C: 9
[1, 5, 9]

--------------------------------------------------------

Calculating relative liabilities for each node...


----------------Liabilities for Node A------------------

Liability of Node A to Node B (i.e. P_ij) is 6.0
Liability of Node A to Node C (i.e. P_ij) is 5.0
Total nominal liabilities for Node A (i.e. p_hat_i) is 11.0

Relative Liability of Node A to Node B is 0.5454545454545454
Relative Liability of Node A to Node C is 0.45454545454545453
Sum of Relative Liabilities for Node A is 1.0

----------------Liabilities for Node B------------------

Liability of Node B to Nod

,A,B,C
A,0.000000,0.545455,0.454545
B,0.400000,0.000000,0.600000
C,0.666667,0.333333,0.000000



--------------------------------------------------------


RELATIVE LIABILITY MATRIX TRANSPOSED Data Frame (i.e. what node i expects to receive from j in relative terms):


,A,B,C
A,0.000000,0.4,0.666667
B,0.545455,0.0,0.333333
C,0.454545,0.6,0.000000


Node A expects to receive 0.4 from Node B
Node A expects to receive 0.6666666666666666 from Node C


Node B expects to receive 0.5454545454545454 from Node A
Node B expects to receive 0.3333333333333333 from Node C


Node C expects to receive 0.45454545454545453 from Node A
Node C expects to receive 0.6 from Node B



--------------------------------------------------------


TOTAL OBLIGATION VECTOR (i.e. total nominal obligations for each node i.e. p_hat_i):
Total nominal obligation for Node A (i.e. p_hat_1): 11.0
Total nominal obligation for Node B (i.e. p_hat_2): 15.0
Total nominal obligation for Node C (i.e. p_hat_3): 6.0
{1: [11.0, 15.0, 6.0]}


In [5]:

######### ALGORITHM ######### 
# Determine each node's payout assumming all other nodes meet their obligations

# If, under the assumption that all nodes pay fully, it is, in fact, the case that all obligations are satisfied, 
# then terminate the algorithm.

# If some nodes default even when all other nodes pay, try to solve the system again, assuming that only these 
# "first-order" defaults occur.

# If only first-order defaults occur under the new clearing vector, then terminate the algorithm.

# If second-order defaults occur, then try to clear again assuming only second-order defaults occur, and so on.

# It is clear that since there are only n nodes, this process must terminate after n iterations. The point at which 
# a node defaults under the algorithm is a measure of the node's exposure to the systemic risks faced by the 
# clearing system.

######### ALGORITHM ######### 


